In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision.models import vgg16
from torchvision import models

# 定义数据预处理操作
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
#     transforms.RandomCrop(32, padding=4),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# 加载 CIFAR-10 数据集
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [2]:
model = models.resnet101(pretrained=True)

In [3]:
model.fc = nn.Linear(2048, 10)

In [4]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4) # 下次测试一下 weight_decay=5e-4
# lr_scheduler = optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=[lr_scheduler])
lr_scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=400)
# 训练模型
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [5]:
# 定义辅助函数计算正确率
def calculate_accuracy(outputs, labels):
    _, predicted = torch.max(outputs, 1)
    total = labels.size(0)
    correct = (predicted == labels).sum().item()
    accuracy = correct / total * 100
    return accuracy

In [6]:
best_acc = .9

In [7]:
# 开始训练
for epoch in range(300):

    running_loss = 0.0
    train_accuracy = 0.0
    s = 0
    acc = 0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        train_accuracy = calculate_accuracy(outputs, labels)
        _, predicted = torch.max(outputs, 1)
        acc += (predicted == labels).sum().item()
        s += len(outputs)
        if (i+1) % 20 == 0:
            print('Epoch=%d Step[%3d/%d] loss: %.3f, train accuracy: %.2f %%' %
              (epoch + 1, i + 1, len(trainloader), loss.item(), train_accuracy))
    print("Epoch = %d, Trainning accuracy = %.2f%%"%(epoch+1, acc / s * 100))
    test_accuracy = 0.0
    total_test_samples = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            test_accuracy += calculate_accuracy(outputs, labels)
            total_test_samples += labels.size(0)
    print('Epoch = %d, Testing accuracy: %.2f %%' % (epoch + 1, test_accuracy / total_test_samples * 100))
    lr_scheduler.step()
    if (test_accuracy / total_test_samples * 100) > best_acc:
        best_acc = test_accuracy / total_test_samples * 100
        torch.save(model.state_dict(), f"resnet101-224-cifar10-{best_acc:.2f}.pth")

Epoch=1 Step[ 20/391] loss: 0.535, train accuracy: 85.16 %
Epoch=1 Step[ 40/391] loss: 0.270, train accuracy: 89.06 %
Epoch=1 Step[ 60/391] loss: 0.350, train accuracy: 86.72 %
Epoch=1 Step[ 80/391] loss: 0.209, train accuracy: 93.75 %
Epoch=1 Step[100/391] loss: 0.283, train accuracy: 90.62 %
Epoch=1 Step[120/391] loss: 0.220, train accuracy: 92.19 %
Epoch=1 Step[140/391] loss: 0.183, train accuracy: 96.09 %
Epoch=1 Step[160/391] loss: 0.143, train accuracy: 96.09 %
Epoch=1 Step[180/391] loss: 0.162, train accuracy: 91.41 %
Epoch=1 Step[200/391] loss: 0.132, train accuracy: 95.31 %
Epoch=1 Step[220/391] loss: 0.207, train accuracy: 92.97 %
Epoch=1 Step[240/391] loss: 0.244, train accuracy: 92.97 %
Epoch=1 Step[260/391] loss: 0.176, train accuracy: 95.31 %
Epoch=1 Step[280/391] loss: 0.120, train accuracy: 96.09 %
Epoch=1 Step[300/391] loss: 0.114, train accuracy: 96.09 %
Epoch=1 Step[320/391] loss: 0.202, train accuracy: 92.97 %
Epoch=1 Step[340/391] loss: 0.060, train accuracy: 98.44

Epoch=7 Step[360/391] loss: 0.036, train accuracy: 98.44 %
Epoch=7 Step[380/391] loss: 0.021, train accuracy: 99.22 %
Epoch = 7, Trainning accuracy = 99.30%
Epoch = 7, Testing accuracy: 96.22 %
Epoch=8 Step[ 20/391] loss: 0.002, train accuracy: 100.00 %
Epoch=8 Step[ 40/391] loss: 0.003, train accuracy: 100.00 %
Epoch=8 Step[ 60/391] loss: 0.003, train accuracy: 100.00 %
Epoch=8 Step[ 80/391] loss: 0.013, train accuracy: 99.22 %
Epoch=8 Step[100/391] loss: 0.003, train accuracy: 100.00 %
Epoch=8 Step[120/391] loss: 0.007, train accuracy: 100.00 %
Epoch=8 Step[140/391] loss: 0.034, train accuracy: 99.22 %
Epoch=8 Step[160/391] loss: 0.017, train accuracy: 99.22 %
Epoch=8 Step[180/391] loss: 0.007, train accuracy: 100.00 %
Epoch=8 Step[200/391] loss: 0.002, train accuracy: 100.00 %
Epoch=8 Step[220/391] loss: 0.003, train accuracy: 100.00 %
Epoch=8 Step[240/391] loss: 0.004, train accuracy: 100.00 %
Epoch=8 Step[260/391] loss: 0.016, train accuracy: 100.00 %
Epoch=8 Step[280/391] loss: 0


KeyboardInterrupt

